# Kamodo SWMF-GM 3D simulation output reader

This reader requires some other packages to be installed.

pip install requests

conda install -c plotly plotly-orca

In [1]:
import kamodo
from readers.swmf_gm import SWMF_GM
from readers.swmf_gm import show_GM_files

import glob
import numpy as np
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected = True)

This unreleased version of SpacePy is not supported by the SpacePy team.
Qin-Denton/OMNI2 data not found in current format. This module has limited functionality.
Run spacepy.toolbox.update(QDomni=True) to download data


In [2]:
runpath = "../../../data/"
runname = "weili_jiang_031012_4"
file = show_GM_files(runpath, runname)

Routine prints simulation data/time for each simulation output and returns last file.
  If year is 1000, then no start_time for the run was found
 and arguement should be added to call.
... found run start_time =  2003/10/28 00:00

2003/10/29 23:57:00 UT     ../../../data/weili_jiang_031012_4/IO2/3d__ful_1_t00475700_n2315109.out
2003/10/31 23:59:00 UT     ../../../data/weili_jiang_031012_4/IO2/3d__ful_1_t00955900_n4632237.out



In [3]:
# Select the data file
runpath = "../../../data/"
runname = "weili_jiang_031012_4"
files = glob.glob(runpath+runname+'/IO2/3d*.out')
files.sort()
file = files[-1]

model = SWMF_GM(file, runpath=runpath, runname=runname)
#model

opening ../../../data/weili_jiang_031012_4/IO2/3d__ful_1_t00955900_n4632237.out
... found 15 variables
... found run start_time =  2003/10/28 00:00
... setting filetime =  2003/10/31 23:59:00 UT
... file grid has 755136 cells with minimum dx = 0.125
Time loading file and precomputing default interpolations: 19.8630 seconds


In [ ]:
model.set_plot('XY')
fig = model.get_plot('p')
iplot(fig)

In [ ]:
model.set_plot('XZ')

In [ ]:
fig = model.get_plot('p')
iplot(fig)

In [ ]:
# This figure can be saved as a PNG, self-contained interactable html file, and a DIV to include in a web page.
#fig.write_image("Yslice.png")
#fig.write_html("Yslice.html")
#fig.write_html("Yslice.div",full_html=False)

In [ ]:
model.set_plot('XYZ')

In [ ]:
fig = model.get_plot('p', colorscale="Rainbow")
iplot(fig)

In [ ]:
fig.update_traces(hovertemplate="X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<br>contour min/max %{data.cmin:.4f} / %{data.cmax:.4f}")
iplot(fig)

In [4]:
model.set_plot('YZ', cutV = 0.)

Time resetting plot and precomputing interpolations: 8.0857 seconds


In [6]:
fig = model.get_plot('ux', colorscale="BlueRed", sym="T")
iplot(fig)

# Interpolation can be imprecise, see example below:

In [ ]:
help(model.GMinterpolate)

In [ ]:
a=np.array([[10.,-3.,0.]], dtype=np.float32)
print(a,a.shape)
b=model.p(a)
print('result is ',b,' shape = ',b.shape)

In [ ]:
a=np.array([[10.,-3.,0.],[10.,-4.,0.],[10.,-3.5,0.]], dtype=np.float32)
print(a,a.shape)
b=model.p(a)
print('result is ',b,' shape = ',b.shape)